In [1]:
!python --version

Python 3.6.9 :: Anaconda, Inc.


In [2]:
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate

Using TensorFlow backend.


In [3]:
from model import create_model
from align import AlignDlib
import os
import numpy as np
import cv2
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, accuracy_score


In [4]:
# Model for 128 embeddings from a image
nn4_small2_pretrained = create_model()
nn4_small2_pretrained.load_weights('weights/nn4.small2.v1.h5')

In [5]:
# Model Summary
nn4_small2_pretrained.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 102, 102, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 48, 48, 64)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 48, 48, 64)   256         conv1[0][0]                      
____________________________________________________________________________________________

In [6]:
# Initialize the OpenFace face alignment utility
alignment = AlignDlib('weights/landmarks.dat')

In [7]:
# Funtion to align an image using dlib utility
def align_image(img):
    return alignment.align(96, img, alignment.getLargestFaceBoundingBox(img), 
                           landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)

In [8]:
def load_image(path):
    img = cv2.imread(path, 1)
    # OpenCV loads images with color channels
    # in BGR order. So we need to reverse them
    return img[...,::-1]


In [9]:
class IdentityMetadata():
    def __init__(self, base, name, file):
        # dataset base directory
        self.base = base
        # identity name
        self.name = name
        # image file name
        self.file = file

    def __repr__(self):
        return self.image_path()

    def image_path(self):
        return os.path.join(self.base, self.name, self.file) 
    
def load_metadata(path):
    metadata = []
    for i in sorted(os.listdir(path)):
        for f in sorted(os.listdir(os.path.join(path, i))):
            # Check file extension. Allow only jpg/jpeg' files.
            ext = os.path.splitext(f)[1]
            print(ext)
            if ext == '.jpg' or ext == '.jpeg':
#                 print(i," ",path," ",f)
                metadata.append(IdentityMetadata(path, i, f))
    return np.array(metadata)

metadata = load_metadata('images')
print(metadata)

.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
.jpg
[images\Aastha_Agrawal\IMG-20191004-WA0015.jpg
 images\Aastha_Agrawal\IMG-20191004-WA0016.jpg
 images\Aastha_Agrawal\IMG-20191004-WA0017.jpg
 images\Aastha_Agrawal\IMG-20191004-WA0018.jpg
 images\Aastha_Agrawal\IMG-20191004-WA0019.jpg
 images\Ariel_Sharon\Ariel_Sharon_0001.jpg
 images\Ariel_Sharon\Ariel_Sharon_0002.jpg
 images\Ariel_Sharon\Ariel_Sharon_0003.jpg
 images\Ariel_Sharon\Ariel_Sharon_0004.jpg
 images\

In [10]:
# print(enumerate(metadata))

# To store image embeddings
embedded = np.zeros((metadata.shape[0], 128))

# Process image through the neural net to get 128 embeddings per image
for i, m in enumerate(metadata):    
    print(i, " ",m)
    img = load_image(m.image_path())
    img = align_image(img)
    # scale RGB values to interval [0,1]
    img = (img / 255.).astype(np.float32)
    # obtain embedding vector for image
    embedded[i] = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]

0   images\Aastha_Agrawal\IMG-20191004-WA0015.jpg

1   images\Aastha_Agrawal\IMG-20191004-WA0016.jpg
2   images\Aastha_Agrawal\IMG-20191004-WA0017.jpg
3   images\Aastha_Agrawal\IMG-20191004-WA0018.jpg
4   images\Aastha_Agrawal\IMG-20191004-WA0019.jpg
5   images\Ariel_Sharon\Ariel_Sharon_0001.jpg
6   images\Ariel_Sharon\Ariel_Sharon_0002.jpg
7   images\Ariel_Sharon\Ariel_Sharon_0003.jpg
8   images\Ariel_Sharon\Ariel_Sharon_0004.jpg
9   images\Ariel_Sharon\Ariel_Sharon_0005.jpg
10   images\Ariel_Sharon\Ariel_Sharon_0006.jpg
11   images\Ariel_Sharon\Ariel_Sharon_0007.jpg
12   images\Ariel_Sharon\Ariel_Sharon_0008.jpg
13   images\Ariel_Sharon\Ariel_Sharon_0009.jpg
14   images\Ariel_Sharon\Ariel_Sharon_0010.jpg
15   images\Arnold_Schwarzenegger\Arnold_Schwarzenegger_0001.jpg
16   images\Arnold_Schwarzenegger\Arnold_Schwarzenegger_0002.jpg
17   images\Arnold_Schwarzenegger\Arnold_Schwarzenegger_0003.jpg
18   images\Arnold_Schwarzenegger\Arnold_Schwarzenegger_0004.jpg
19   images\Arnold_Schwa

In [11]:
embedded.shape

(117, 128)

In [12]:
from sklearn.preprocessing import LabelEncoder
targets = np.array([m.name for m in metadata])
print(targets)
encoder = LabelEncoder()
encoder.fit(targets)

# Numerical encoding of identities
y = encoder.transform(targets)


['Aastha_Agrawal' 'Aastha_Agrawal' 'Aastha_Agrawal' 'Aastha_Agrawal'
 'Aastha_Agrawal' 'Ariel_Sharon' 'Ariel_Sharon' 'Ariel_Sharon'
 'Ariel_Sharon' 'Ariel_Sharon' 'Ariel_Sharon' 'Ariel_Sharon'
 'Ariel_Sharon' 'Ariel_Sharon' 'Ariel_Sharon' 'Arnold_Schwarzenegger'
 'Arnold_Schwarzenegger' 'Arnold_Schwarzenegger' 'Arnold_Schwarzenegger'
 'Arnold_Schwarzenegger' 'Arnold_Schwarzenegger' 'Arnold_Schwarzenegger'
 'Arnold_Schwarzenegger' 'Arnold_Schwarzenegger' 'Arnold_Schwarzenegger'
 'Colin_Powell' 'Colin_Powell' 'Colin_Powell' 'Colin_Powell'
 'Colin_Powell' 'Colin_Powell' 'Colin_Powell' 'Colin_Powell'
 'Colin_Powell' 'Colin_Powell' 'Dev_Pabreja' 'Dev_Pabreja' 'Dev_Pabreja'
 'Dev_Pabreja' 'Dev_Pabreja' 'Dev_Pabreja' 'Donald_Rumsfeld'
 'Donald_Rumsfeld' 'Donald_Rumsfeld' 'Donald_Rumsfeld' 'Donald_Rumsfeld'
 'Donald_Rumsfeld' 'Donald_Rumsfeld' 'Donald_Rumsfeld' 'Donald_Rumsfeld'
 'Donald_Rumsfeld' 'George_W_Bush' 'George_W_Bush' 'George_W_Bush'
 'George_W_Bush' 'George_W_Bush' 'George_W_Bush' 

In [13]:
# train_idx = np.arange(metadata.shape[0]) % 2 != 0
# test_idx = np.arange(metadata.shape[0]) % 2 == 0

# # 50 train examples of 10 identities (5 examples each)
# X_train = embedded[train_idx]
# # 50 test examples of 10 identities (5 examples each)
# X_test = embedded[test_idx]

# y_train = y[train_idx]
# y_test = y[test_idx]

# svc = SVC(probability=True)

# svc.fit(X_train, y_train)

# acc_svc = accuracy_score(y_test, svc.predict(X_test))

# print(f'SVM accuracy = {acc_svc}')

In [14]:
train_idx = np.arange(metadata.shape[0]) 

# 50 train examples of 10 identities (5 examples each)
X_train = embedded[train_idx]

y_train = y[train_idx]

svc = KNeighborsClassifier()

svc.fit(X_train, y_train)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [15]:
# Prediction Code for live Images

In [16]:
font = cv2.FONT_HERSHEY_SIMPLEX

def processFrame(img):
    # Change RGB to BGR for imshow
    origImg = img[...,::-1]
    
    # Get bounding box
    bb = alignment.getLargestFaceBoundingBox(img)
    
    # Align image
    img = align_image(img)
    
    # scale RGB values to interval [0,1]
    img = (img / 255.).astype(np.float32)
    
    # obtain embedding vector for image
    features = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]
    
    # Get the class for prediction
    predClass = svc.predict([features])
    predClass = encoder.inverse_transform(predClass)[0]
    
    # Display the image with bounding box & label
    cv2.putText(origImg,predClass,(bb.left(),bb.top()-5), font, 0.5, (0,0,255), 1, cv2.LINE_AA)
    cv2.rectangle(origImg,(bb.left(),bb.top()),(bb.right(),bb.bottom()),(0,0,255),2)
    cv2.imshow('image',origImg)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [21]:
font = cv2.FONT_HERSHEY_SIMPLEX

def processFrame1(img):
    # Change RGB to BGR for imshow
    origImg = img[...,::-1]
    
    # Get bounding box
    bb = alignment.getLargestFaceBoundingBox(img)
    
    # Align image
    img = align_image(img)
    
    # scale RGB values to interval [0,1]
    img = (img / 255.).astype(np.float32)
    
    # obtain embedding vector for image
    features = nn4_small2_pretrained.predict(np.expand_dims(img, axis=0))[0]
    
    # Get the class for prediction
    predScores = svc.predict_proba([features])
    print(predScores)
    predClass = None
    
    maxProb = (predScores[0]).max()
    
    print(maxProb)
    
    if(maxProb > 0.6):
    
        predClass = [(predScores[0]).argmax()]
        print(predClass)
        predClass = encoder.inverse_transform(predClass)[0]
    
    
    print(predClass)
    
    return bb, predClass

In [18]:
img1 = load_image("IMG-20191004-WA0009.jpg")
processFrame1(img1)

[[0.  0.  0.  0.  0.8 0.  0.  0.  0.  0.  0.2 0.  0. ]]
0.8
[4]
Dev_Pabreja


(rectangle(277,598,598,919), 'Dev_Pabreja')

In [22]:
students = []

In [23]:
cap = cv2.VideoCapture(0)
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if(alignment.getLargestFaceBoundingBox(frame)!=None):
    
        bb, predClass = processFrame1(frame)

        cv2.rectangle(frame,(bb.left(),bb.top()),(bb.right(),bb.bottom()),(0,0,255),2)
        if(predClass!=None):
            cv2.putText(frame,predClass,(bb.left(),bb.top()-5), font, 0.5, (0,0,255), 1, cv2.LINE_AA)
            
            if predClass not in students:
                students.append(predClass)
        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
1.0
[0]
Aastha_Agrawal
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
1.0
[0]
Aastha_Agrawal
[[0.  0.  0.2 0.4 0.4 0.  0.  0.  0.  0.  0.  0.  0. ]]
0.4
None
[[0.  0.  0.  0.4 0.2 0.  0.  0.  0.  0.  0.2 0.2 0. ]]
0.4
None
[[0.  0.  0.  0.4 0.6 0.  0.  0.  0.  0.  0.  0.  0. ]]
0.6
None
[[0.  0.  0.  0.2 0.8 0.  0.  0.  0.  0.  0.  0.  0. ]]
0.8
[4]
Dev_Pabreja
[[0.  0.  0.  0.4 0.6 0.  0.  0.  0.  0.  0.  0.  0. ]]
0.6
None
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
1.0
[4]
Dev_Pabreja
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
1.0
[4]
Dev_Pabreja
[[0.  0.  0.  0.4 0.4 0.  0.  0.  0.  0.  0.  0.2 0. ]]
0.4
None
[[0.  0.  0.  0.  0.8 0.  0.  0.  0.  0.  0.2 0.  0. ]]
0.8
[4]
Dev_Pabreja
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
1.0
[4]
Dev_Pabreja
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
1.0
[4]
Dev_Pabreja
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
1.0
[4]
Dev_Pabreja
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
1.0
[4]
Dev_Pabreja
[[0.  0.  0. 